In [3]:
import pandas as pd
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm 
from urllib.parse import urljoin


## Initial data

In [123]:
db_pomoci = pd.read_csv("db_pomoci.csv")
black_list = ['http://www.dc-brno.cz']
df = db_pomoci.rename(columns={"Webová stránka" : "web", "Název":"nazev"})
df = df[(~df["web"].isna()) & (~df['web'].isin(black_list))]
df.loc[df['web'].str.startswith('www'), 'web'] = df['web'].str.replace('^www', 'https://www', regex=True)
# df['web'] = df['web'].str.replace('https://www', 'www')
# df['web'] = df['web'].str.replace('www', 'https://www')
df_agg = df.groupby("web")["Telefon"].agg(list)
df_agg["email"] = df.groupby("web")["E-mail"].agg(list)
df_agg = df_agg.reset_index().rename(columns={'web': 'web', 'Telefon': 'Telefon'})
df_agg = pd.DataFrame(df_agg)

# Scraped data

db_scraped = pd.read_csv("data/29_05/df_scraped_v2.csv")
df_res = db_scraped[~db_scraped["Phone Numbers"].isna()]
df_res = df_res.rename(columns={'web': 'web', 'Telefon': 'Telefon'})#.drop("Unnamed: 0", axis= 1)
df_res = df_res.rename(columns= {"Website":"web"})
df_res = df_res.rename(columns= {"Phone Numbers":"phones_scraped"})
df_res = df_res.rename(columns= {"Emails":"emails_scraped"})


maps_df = pd.read_csv("data/maps_results.csv",sep=";")
maps_df = maps_df.rename(columns={'Web': 'web', 'Phone': 'Telefon Mapy'})
mapy = maps_df[["web","Telefon Mapy"]]#,"Telefon Mapy"


In [124]:
print("df_agg", df_agg.shape)
print("df_res", df_res.shape)
print("df_mapy", mapy.shape)



df_agg (871, 2)
df_res (599, 3)
df_mapy (1456, 2)


In [125]:
df_res['phones_scraped'] = df_res['phones_scraped'].str.split(', ')
mapy['Telefon Mapy'] = mapy['Telefon Mapy'].str.split(', ')
df_res = df_res[["web","phones_scraped"]]
df_res['source'] = 'scraped'
df_agg['source'] = 'db'
mapy['source'] = 'mapy.cz'
#df_res['emails_scraped'] = df_res['emails_scraped'].str.split(', ')
#df_agg = df_agg.rename(columns= {"Telefon":"phones_scraped"})
df_res_exp = df_res.explode('phones_scraped').reset_index(drop=True)
mapy_exp = mapy.explode('Telefon Mapy').reset_index(drop=True)
mapy_exp = mapy_exp.rename(columns= {"Telefon Mapy":"Telefon"})
df_res_exp = df_res.rename(columns= {"phones_scraped":"Telefon"})
df_agg_exp = df_agg.explode('Telefon').reset_index(drop=True)
df_union = pd.concat([df_agg_exp, df_res_exp,mapy_exp])

/var/folders/rd/l77bltt96_97jdz4bbkmq5z40000gn/T/ipykernel_46796/1908990668.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapy['Telefon Mapy'] = mapy['Telefon Mapy'].str.split(', ')
/var/folders/rd/l77bltt96_97jdz4bbkmq5z40000gn/T/ipykernel_46796/1908990668.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapy['source'] = 'mapy.cz'


In [126]:
mapy_exp

,web,Telefon,source
0,http://www.pdz.cz,778 526 216,mapy.cz
1,http://www.cdzopava.cz/,553 652 433,mapy.cz
2,http://www.cdzopava.cz/,723 457 998,mapy.cz
3,http://www.pdz.cz,469 626 383,mapy.cz
4,http://www.pdz.cz,774 915 755,mapy.cz
...,...,...,...
1634,www.praha.ymca.cz,775 317 814,mapy.cz
1635,http://www.ymkarium.ymca.cz,737 629 149,mapy.cz
1636,www.dixie.ymca.cz,737 872 116,mapy.cz
1637,http://www.usti.ymca.cz,475 220 831,mapy.cz


In [127]:
df_union.groupby("source").count()

,web,Telefon
source,,
db,1986,1971
mapy.cz,1267,1609
scraped,599,599


In [132]:
df_union[df_union["web"]=="www.zvole.info"]

,web,Telefon,source
1533,www.zvole.info,NaN,mapy.cz


In [128]:
set_unmatched = set(df_union[df_union["Telefon"].isna() & df_union["web"]!= 'email' ]["web"])


In [129]:
len(set_unmatched)

1009

In [63]:
#df_union['Telefon'] = df_union['Telefon'].str.replace(r'\s+', '', regex=True)
#df_grouped = pd.DataFrame(df_union.groupby('web').count().sort_values(by="Telefon",ascending=False))

In [130]:
df_grouped = df_union.groupby('web').count()

# Filter the DataFrame to get rows where the count of 'Telefon' is less than 1
filtered_df = df_grouped[df_grouped['Telefon'] < 1]

# Extract the list of 'web' values
web_list = filtered_df.index.tolist()
print(len(web_list))
for c in web_list:
    print(c)

6
http://www.ckp-dobrichovice.cz
http://www.fokus-mb.cz/klubko
https://www.milicak.cz
misa.zapletalova@tiscali.cz
www.milicak.cz
www.zvole.info


In [134]:
#df_union[df_union["web"] == 'asistovanekontakty.cz'].sort_values(by="Telefon")
web = 'https://www.zvole.info'
df_agg[df_agg["web"] == web].sort_values(by="Telefon")
df_res[df_res["web"] == web].sort_values(by="phones_scraped")

,web,phones_scraped,source
761,https://www.zvole.info,"[+420 257 760 337, 257 760 337]",scraped


In [14]:
preview = df_res.merge(df_agg , on="web")
preview = preview.merge(mapy_tel , on="web")
preview['phones_scraped'] = preview['phones_scraped'].str.split(', ')
preview


,web,phones_scraped,Telefon,Telefon Mapy
0,http://www.pdz.cz,"[121 407 9329, 64 24 22 18, 420 466 400 116]","[778 526 216, 469 626 383, 774 915 755]",778 526 216
1,http://www.pdz.cz,"[121 407 9329, 64 24 22 18, 420 466 400 116]","[778 526 216, 469 626 383, 774 915 755]","469 626 383, 774 915 755"
2,http://www.medsix.cz/?option=com_content&view=...,"[483 341 232, 483 300 087, 483 310 919, 483 34...",[483 341 267],483 341 267
3,https://jindrichuvhradec.charita.cz/sluzba/852/,"[731 402 982, 731 604 554, 605 849 877]","[731 402 982, 605 849 877]","731 402 982, 605 849 877"
4,https://www.homeopatiecz.com/,"[777 169 278, 736 660 315, 777 146 760, 725 76...",[736 660 315],736 660 315
...,...,...,...,...
612,https://www.firmy.cz/detail/13179143-phdr-tere...,[420 376 322 688],[376 322 688],376 322 688
613,https://search.seznam.cz/?q=psychiatrick%C3%A1...,[420608971030],[736 132 143],736 132 143
614,http://ssp-ol.cz/,[585 427 141],[774 406 453],774 406 453
615,http://www.psychoterapeuti.cz/adresar-psychote...,"[284 016 451, 736 660 315, 530 334 600, 775 18...",[466 611 601],466 611 601


In [1]:
df_agg.head()

NameError: name 'df_agg' is not defined